In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Concatenate, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
import pandas as pd

prices_df = pd.read_csv("houseinfo.txt", header=None, names=["price"])
prices_df.index += 1

In [3]:
IMG_SIZE = (224, 224)
image_types = ["bathroom", "bedroom", "kitchen", "frontal"]
image_dir = "Houses Dataset"


In [4]:

def load_images_and_labels(image_dir, prices_df):
    images = {img_type: [] for img_type in image_types}
    labels = []

    for index, row in prices_df.iterrows():
        house_price = row["price"]
        house_images = {}

        for img_type in image_types:
            img_path = os.path.join(image_dir, f"{index}_{img_type}.jpg")
            if os.path.exists(img_path):
                img = load_img(img_path, target_size=IMG_SIZE)
                img = img_to_array(img) / 255.0  
                house_images[img_type] = img

        if len(house_images) == 4:
            for img_type in image_types:
                images[img_type].append(house_images[img_type])
            labels.append(house_price)

    return {img_type: np.array(images[img_type]) for img_type in image_types}, np.array(labels)

In [5]:
X, y = load_images_and_labels(image_dir, prices_df)

In [6]:
scaler = MinMaxScaler()
y = scaler.fit_transform(y.reshape(-1, 1))

In [7]:
X_train, X_val, y_train, y_val = {}, {}, {}, {}

In [8]:

for img_type in image_types:
    X_train[img_type], X_val[img_type], y_train, y_val = train_test_split(X[img_type], y, test_size=0.2, random_state=42)

In [9]:
def build_resnet50_branch(input_shape):
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=input_shape)
    base_model.trainable = False
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.3)(x)
    return Model(base_model.input, x)

In [10]:
inputs = {img_type: Input(shape=(224, 224, 3), name=img_type) for img_type in image_types}

In [11]:
branches = {img_type: build_resnet50_branch((224, 224, 3))(inputs[img_type]) for img_type in image_types}

In [12]:
merged = Concatenate()(list(branches.values()))
output = Dense(1, activation="linear")(merged)

In [13]:
model = Model(inputs=list(inputs.values()), outputs=output)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ bathroom            │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bedroom             │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ kitchen             │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ frontal             │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional          │ (None, 128)       │ 23,849,984 │ bathroom[0][0]    │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_1        │ (None, 128)       │ 23,849,984 │ bedroom[0][0]     │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_2        │ (None, 128)       │ 23,849,984 │ kitchen[0][0]     │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_3        │ (None, 128)       │ 23,849,984 │ frontal[0][0]     │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 512)       │          0 │ functional[0][0], │
│ (Concatenate)       │                   │            │ functional_1[0][… │
│                     │                   │            │ functional_2[0][… │
│                     │                   │            │ functional_3[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 1)         │        513 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 95,400,449 (363.92 MB)

 Trainable params: 1,049,601 (4.00 MB)

 Non-trainable params: 94,350,848 (359.92 MB)

In [14]:
early_stop = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint("house_price_model.h5", save_best_only=True, monitor="val_loss")
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stop, model_checkpoint],
    verbose=1
)

Epoch 1/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - loss: 2.2122 - mae: 1.2325 

14/14 ━━━━━━━━━━━━━━━━━━━━ 339s 22s/step - loss: 2.1736 - mae: 1.2200 - val_loss: 0.3971 - val_mae: 0.6270
Epoch 2/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - loss: 0.6247 - mae: 0.6513 

14/14 ━━━━━━━━━━━━━━━━━━━━ 268s 19s/step - loss: 0.6141 - mae: 0.6443 - val_loss: 0.1483 - val_mae: 0.3799
Epoch 3/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - loss: 0.2267 - mae: 0.3822 

14/14 ━━━━━━━━━━━━━━━━━━━━ 232s 17s/step - loss: 0.2256 - mae: 0.3813 - val_loss: 0.0290 - val_mae: 0.1580
Epoch 4/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - loss: 0.1395 - mae: 0.3014 

14/14 ━━━━━━━━━━━━━━━━━━━━ 229s 17s/step - loss: 0.1394 - mae: 0.3008 - val_loss: 0.0042 - val_mae: 0.0480
Epoch 5/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 254s 16s/step - loss: 0.1269 - mae: 0.2836 - val_loss: 0.0138 - val_mae: 0.1060
Epoch 6/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 225s 16s/step - loss: 0.0702 - mae: 0.2088 - val_loss: 0.0122 - val_mae: 0.0994
Epoch 7/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 225s 16s/step - loss: 0.0585 - mae: 0.1928 - val_loss: 0.0083 - val_mae: 0.0705
Epoch 8/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 255s 16s/step - loss: 0.0535 - mae: 0.1871 - val_loss: 0.0064 - val_mae: 0.0600
Epoch 9/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - loss: 0.0361 - mae: 0.1447 

14/14 ━━━━━━━━━━━━━━━━━━━━ 228s 16s/step - loss: 0.0359 - mae: 0.1441 - val_loss: 0.0041 - val_mae: 0.0477
Epoch 10/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - loss: 0.0228 - mae: 0.1225 

14/14 ━━━━━━━━━━━━━━━━━━━━ 336s 22s/step - loss: 0.0230 - mae: 0.1227 - val_loss: 0.0041 - val_mae: 0.0470
Epoch 11/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 274s 18s/step - loss: 0.0231 - mae: 0.1092 - val_loss: 0.0071 - val_mae: 0.0738
Epoch 12/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 231s 16s/step - loss: 0.0179 - mae: 0.1032 - val_loss: 0.0080 - val_mae: 0.0688
Epoch 13/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 223s 16s/step - loss: 0.0175 - mae: 0.0958 - val_loss: 0.0061 - val_mae: 0.0655
Epoch 14/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 225s 16s/step - loss: 0.0131 - mae: 0.0890 - val_loss: 0.0052 - val_mae: 0.0578
Epoch 15/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 230s 17s/step - loss: 0.0099 - mae: 0.0753 - val_loss: 0.0042 - val_mae: 0.0487
Epoch 16/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 248s 18s/step - loss: 0.0100 - mae: 0.0693 - val_loss: 0.0043 - val_mae: 0.0493
Epoch 17/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 222s 16s/step - loss: 0.0108 - mae: 0.0648 - val_loss: 0.0043 - val_mae: 0.0493
Epoch 18/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - loss: 0

14/14 ━━━━━━━━━━━━━━━━━━━━ 318s 20s/step - loss: 0.0090 - mae: 0.0643 - val_loss: 0.0039 - val_mae: 0.0462
Epoch 19/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 319s 23s/step - loss: 0.0129 - mae: 0.0680 - val_loss: 0.0042 - val_mae: 0.0499
Epoch 20/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 274s 19s/step - loss: 0.0094 - mae: 0.0619 - val_loss: 0.0040 - val_mae: 0.0471
Epoch 21/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 325s 19s/step - loss: 0.0062 - mae: 0.0542 - val_loss: 0.0042 - val_mae: 0.0477
Epoch 22/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 265s 19s/step - loss: 0.0075 - mae: 0.0546 - val_loss: 0.0041 - val_mae: 0.0481
Epoch 23/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 265s 19s/step - loss: 0.0084 - mae: 0.0571 - val_loss: 0.0042 - val_mae: 0.0488
Epoch 24/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 318s 19s/step - loss: 0.0074 - mae: 0.0566 - val_loss: 0.0041 - val_mae: 0.0491
Epoch 25/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 258s 19s/step - loss: 0.0062 - mae: 0.0529 - val_loss: 0.0042 - val_mae: 0.0495
Epoch 26/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 257s 19s/step - loss:

In [15]:
model.save("house_price_model.h5")
test_loss, test_mae = model.evaluate(X_val, y_val)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 54s 12s/step - loss: 0.0041 - mae: 0.0482
Test Loss: 0.0038884433452039957, Test MAE: 0.04622309282422066


In [16]:
def predict_house_price(image_paths):
    house_images = {}

    for img_type, img_path in zip(image_types, image_paths):
        img = load_img(img_path, target_size=IMG_SIZE)
        img = img_to_array(img) / 255.0
        house_images[img_type] = np.expand_dims(img, axis=0)

    predicted_price = model.predict(house_images)
    return scaler.inverse_transform(predicted_price)[0][0]

In [17]:
test_images=[
    "Houses Dataset/1_bathroom.jpg",
    "Houses Dataset/1_bedroom.jpg",
    "Houses Dataset/1_kitchen.jpg",
    "Houses Dataset/1_frontal.jpg"
]
predicted_prices = predict_house_price(test_images)
print(f"Predicted Price: {predicted_prices}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 30s 30s/step
Predicted Price: 9123830.0
